In [1]:
from CONST import *
import music21
import glob

In [2]:
# Convert files in directory
files = glob.glob("TEST\**\*.mid", recursive=True)
len(files)

25

In [7]:
def analyzeFile(i):
    f = files[i] 

    # first, transpose the entire score into C major or A minor
    score = music21.converter.parse(f)
    key = score.analyze('key')
    mode = key.mode

    if mode == "major":
        halfSteps = majors[key.tonic.name]

    elif mode == "minor":
        halfSteps = minors[key.tonic.name]

    newscore = score.transpose(halfSteps)

    ### Now, print out score in our text format ###

    # Major/Minor
    print(mode)

    # Print parts
    for part in score.parts:
        # Find most common note
        l = [e.pitch for e in part.notes if e.isNote]
        c = [e.pitches[0] for e in part.notes if e.isChord]
        L = l+c

        # somehow have no elements in this part...
        if len(L) == 0:
            return

        m = max(set(L), key=L.count)

        print(START_CODE)

        # case on the most common pitch
        if m < music21.pitch.Pitch(MIDDLENOTE):
            print(BASS_MARK)
        else:
            print(SOPRANO_MARK)

        # prints out the notes
        for e in part.notesAndRests:
            d = float(e.duration.quarterLength)
            
            # identify the closest note length according to our defined durations
            d = NOTE_DURATIONS.index(min(NOTE_DURATIONS, key=lambda x:abs(x-d)))
            
            # CHORD. Encoded in base 12.
            if e.isChord:
                tempLen = min(3, len(e.pitches))
                
                n = 0
                
                notes = []
                
                for i in range(0, tempLen):
                    currNote = e.pitches[i]
                    
                    notes.append(int(currNote.ps))
                    
                # sort from lowest to highest
                notes.sort()
                
                for i in range(0, tempLen):
                    n += (notes[i] % OCTAVE_RANGE) * pow(OCTAVE_RANGE, i)
            
                # we MUST offset in order to account for notes and rests
                n += CHORD_OFFSET
                
            # NOTE. Encoded from 0 (c0) to 95 (b7)
            elif e.isNote:
                n = int(e.pitch.ps) - LOWEST_NOTE_OFFSET
                
                if n < 0:
                    n = n % NOTE_RANGE
                elif n >= NOTE_RANGE:
                    n = n % OCTAVE_RANGE + HIGHEST_C
                
            # REST. Just a special rest marker
            else:
                n = REST_NUM
                
            print("{0} {1}".format(n, d))
                
        print(END_CODE)

In [12]:
analyzeFile(9)

major
S
H
64 6
59 5
56 5
64 5
59 5
56 5
64 13
65 6
60 5
57 5
65 5
60 5
57 5
65 13
67 6
62 5
59 5
67 5
62 5
59 5
67 11
69 5
69 5
100 5
71 13
E
S
L
200 13
200 7
200 4
200 4
200 5
200 5
100 5
213 13
213 7
213 4
213 4
213 5
213 5
100 5
239 5
186 5
47 5
239 5
186 5
47 5
239 7
239 4
239 4
121 5
121 5
100 5
147 13
E
